In [ ]:
import random
import numpy as np

def generar_solucion_aleatoria(n):
    """
    Genera una permutación aleatoria de tamaño n.
    Basado en la definición del espacio de búsqueda |Ω| = n!.
    """
    elementos = list(range(n))
    
    random.shuffle(elementos)
    
    return elementos

def calcular_fitness(matriz, permutacion):
    """
    Calcula la suma de las entradas por encima de la diagonal principal
    dada una permutación específica de filas y columnas.
    Objetivo: Maximizar esta suma.
    """
    n = len(permutacion)
    
    fitness = 0
    
    for i in range(n - 1):
        u = permutacion[i]
        
        for j in range(i + 1, n):
            v = permutacion[j]
            
            valor = matriz[u][v]
            
            fitness += valor
            
    return fitness

# --- ZONA DE EJECUCIÓN ---

if __name__ == "__main__":
    # Definimos una matriz de ejemplo (Entorno estático y determinista )
    matriz_ejemplo = np.array([
        [0, 10, 5, 2],
        [2, 0, 20, 1],
        [4, 1, 0, 15],
        [3, 8, 6, 0]
    ])
    
    n_elementos = len(matriz_ejemplo)

    print(f"--- Prueba de LOP (n={n_elementos}) ---")

    # 1. Generar una solución aleatoria
    mi_permutacion = generar_solucion_aleatoria(n_elementos)
    print(f"Permutación generada: {mi_permutacion}")

    # 2. Calcular su calidad (Fitness)
    mi_fitness = calcular_fitness(matriz_ejemplo, mi_permutacion)
    print(f"Fitness calculado: {mi_fitness}")

--- Prueba de LOP (n=4) ---
Permutación generada: [3, 0, 2, 1]
Fitness calculado: 33
